# Speed Up IGamma

In [26]:
# imports
from importlib import reload

import numpy as np
from scipy.special import gammainc

import mpmath
from zdm import zdm

# Time simple callscipy.special

In [3]:
gamma = -1.1
Emin = 1e31
Emax = 1e42

In [5]:
%timeit -r 10 norm = float(mpmath.gammainc(gamma, a=Emin/Emax))

237 µs ± 7.73 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


# Time the array loop

In [13]:
Eth = 10**np.linspace(41., 43., 100)
Eth_Emax = Eth/Emax

In [14]:
# If this is too slow, we can adopt scipy + recurrance
%timeit -r 10 numer = np.array([float(mpmath.gammainc(gamma, a=iEE)) for iEE in Eth_Emax])

29.5 ms ± 1.32 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


# Real example

In [16]:
params = (1000000000000000000000000000000, 2.5118864315095718e+41, -1.01)

In [17]:
Eth = np.load('Eth.npy')

In [18]:
Eth.shape

(500, 1400)

In [25]:
reload(zdm)
%timeit -r 2  result = zdm.array_cum_gamma(Eth, params[0], params[1], params[2])

1min 49s ± 1.94 s per loop (mean ± std. dev. of 2 runs, 1 loop each)


----

# Vectorize input?

In [28]:
mpmath.gammainc?

Signature: mpmath.gammainc(z, a=0, b=None, regularized=False)
Docstring:
``gammainc(z, a=0, b=inf)`` computes the (generalized) incomplete
gamma function with integration limits `[a, b]`:

.. math ::

  \Gamma(z,a,b) = \int_a^b t^{z-1} e^{-t} \, dt

The generalized incomplete gamma function reduces to the
following special cases when one or both endpoints are fixed:

* `\Gamma(z,0,\infty)` is the standard ("complete")
  gamma function, `\Gamma(z)` (available directly
  as the mpmath function :func:`~mpmath.gamma`)
* `\Gamma(z,a,\infty)` is the "upper" incomplete gamma
  function, `\Gamma(z,a)`
* `\Gamma(z,0,b)` is the "lower" incomplete gamma
  function, `\gamma(z,b)`.

Of course, we have
`\Gamma(z,0,x) + \Gamma(z,x,\infty) = \Gamma(z)`
for all `z` and `x`.

Note however that some authors reverse the order of the
arguments when defining the lower and upper incomplete
gamma function, so one should be careful to get the correct
definition.

If also given the keyword argument ``regularize

In [27]:
mpmath.gammainc(gamma, a=[1., 2.])

TypeError: cannot create mpf from [1.0, 2.0]

----

# Check scipy

----

# Boost

### https://www.boost.org/doc/libs/1_78_0/libs/math/doc/html/math_toolkit/sf_gamma/igamma.html 

# C examples

### https://people.sc.fsu.edu/~jburkardt/c_src/asa239/asa239.html 

# More

### https://scicomp.stackexchange.com/questions/3341/fast-and-accurate-double-precision-implementation-of-incomplete-gamma-function 

# PyGSL

### https://github.com/pygsl/pygsl